# User Management System - CRUD Operations

This notebook implements CRUD operations for a user management system using a relational database. The `users` table contains the following fields:
- `id`: Primary Key
- `name`: String
- `email`: String (unique)
- `created_at`: Timestamp

### Tasks
1. Create a new user.
2. Read user details by ID.
3. Update a user's email.
4. Delete a user by ID.

We'll also discuss the strengths and weaknesses of using a relational database and consider scalability techniques for handling large datasets.


### Step 1: Connect to the MySQL Database

In this step, we establish a connection to the MySQL database where the `users` table will be created and managed. Ensure that the MySQL server is running and the connection details are correct.


In [18]:
import mysql.connector
from datetime import datetime

# Establish the database connection
db = mysql.connector.connect(
    host="localhost",      
    user="root",           
    password="",   
    database="user_management"  # Replace with your database name
)

# Create a cursor object
cursor = db.cursor()

print("Database connection established successfully.")



Database connection established successfully.


### Step 2: Create the `users` Table

Here, we create the `users` table with the following columns:
- `id`: Primary key, auto-incremented.
- `name`: A string representing the user's name.
- `email`: A unique string representing the user's email address.
- `created_at`: A timestamp indicating when the user was created, defaulting to the current timestamp.


In [19]:
# Create the users table
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    email VARCHAR(255) UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

print("Table 'users' created successfully.")


Table 'users' created successfully.


### Step 3: Create a New User

In this step, we implement the function `create_user` to insert a new user into the `users` table. The function takes the user's `name` and `email` as parameters.


In [25]:
def create_user(name, email):
    try:
        # Check if the email already exists
        cursor.execute("SELECT id FROM users WHERE email = %s", (email,))
        result = cursor.fetchone()

        if result:
            print(f"User with email '{email}' already exists with ID: {result[0]}")
        else:
            # Proceed with inserting the new user
            sql = "INSERT INTO users (name, email) VALUES (%s, %s)"
            val = (name, email)
            cursor.execute(sql, val)
            db.commit()
            print(f"User '{name}' created successfully with ID: {cursor.lastrowid}")
    except mysql.connector.Error as err:
        db.rollback()  # Rollback the transaction if there is an error
        print(f"Failed to create user: {err}")

# Example usage
create_user("A.sarath", "sarath@gmail.com")



User 'A.sarath' created successfully with ID: 5


### Step 4: Read User Details by ID

This step implements the function `read_user` to fetch and display the details of a user by their `id`.


In [27]:
# Read user details by ID
def read_user(user_id):
    sql = "SELECT * FROM users WHERE id = %s"
    cursor.execute(sql, (user_id,))
    result = cursor.fetchone()
    if result:
        print(f"User Details: ID={result[0]}, Name={result[1]}, Email={result[2]}, Created At={result[3]}")
    else:
        print(f"No user found with ID: {user_id}")

# Example usage
read_user(0)


No user found with ID: 0


### Step 5: Update User's Email

This step implements the function `update_user_email` to update the email address of a user based on their `id`.


In [30]:
def update_user_email(user_id, new_email):
    try:
        # Check if the new email already exists for a different user
        cursor.execute("SELECT id FROM users WHERE email = %s", (new_email,))
        result = cursor.fetchone()

        if result:
            if result[0] == user_id:
                print(f"New email is the same as the current email for user ID {user_id}.")
            else:
                print(f"Email '{new_email}' is already in use by another user with ID {result[0]}.")
        else:
            # Proceed with updating the user's email
            sql = "UPDATE users SET email = %s WHERE id = %s"
            cursor.execute(sql, (new_email, user_id))
            db.commit()
            if cursor.rowcount > 0:
                print(f"User ID {user_id}'s email updated to {new_email}.")
            else:
                print(f"No user found with ID: {user_id}")
    except mysql.connector.Error as err:
        db.rollback()  # Rollback the transaction if there is an error
        print(f"Failed to update email: {err}")

# Example usage
update_user_email(1, "sankaragraharapu@gmail.com")


User ID 1's email updated to sankaragraharapu@gmail.com.



### Step 6: Delete a User by ID

In this step, we implement the function `delete_user` to remove a user from the `users` table based on their `id`.



In [29]:
# Delete a user by ID
def delete_user(user_id):
    sql = "DELETE FROM users WHERE id = %s"
    cursor.execute(sql, (user_id,))
    db.commit()
    if cursor.rowcount > 0:
        print(f"User ID {user_id} deleted successfully.")
    else:
        print(f"No user found with ID: {user_id}")

# Example usage
delete_user(4)

User ID 4 deleted successfully.


### Step 7: Close the Connection

Finally, we close the database connection to ensure that no resources are left open.


In [ ]:
# Close the cursor and connection
cursor.close()
db.close()

print("Database connection closed.")


# Strengths and Weaknesses of Relational Databases

**Strengths:**
1. **ACID Compliance**: Ensures reliable transactions and data integrity.
2. **Data Integrity**: Strong data integrity through foreign keys, constraints, and normalization.
3. **Complex Queries**: Supports complex SQL queries and joins across multiple tables.
4. **Standardization**: SQL is a widely adopted standard for database queries.

**Weaknesses:**
1. **Scalability**: Vertical scaling can become expensive and challenging for very large datasets.
2. **Rigid Schema**: Schema changes can be difficult and time-consuming.
3. **Performance**: May suffer in write-heavy applications or with large volumes of unstructured data.


# Handling Scalability Beyond 10TB

As the database grows beyond 10TB, traditional relational databases might struggle with performance and management. To handle such large datasets, consider the following techniques:

1. **Partitioning**:
   - **Horizontal Partitioning (Sharding)**: Split large tables across multiple databases or servers. Each shard contains a subset of the data, allowing for parallel processing and improved performance.
   - **Vertical Partitioning**: Split a table into smaller tables containing fewer columns, which can improve performance for specific queries.

2. **Sharding**:
   - Distribute data across multiple servers (shards) based on a shard key, such as `user_id`. This distributes the load and allows for horizontal scaling.

3. **Indexing**:
   - Use indexes on frequently queried columns to improve read performance. However, be mindful of the trade-offs, as excessive indexing can slow down write operations.

4. **Caching**:
   - Implement caching layers (e.g., Redis, Memcached) to store frequently accessed data in memory, reducing the load on the database.

5. **Data Archiving**:
   - Move old or infrequently accessed data to an archive or data warehouse, keeping the active dataset smaller and more manageable.

6. **Database Optimization**:
   - Regularly optimize database queries, normalize tables, and monitor performance to ensure efficient data access and storage.

By employing these techniques, you can scale your relational database to handle larger datasets while maintaining performance and reliability.
